In [1]:
#check if model stays the same
#reimplement data generation

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
from tqdm import tqdm
import random
import numpy as np

DEVICE="cuda"


In [3]:
def set_seed(seed):
    # Set the seed for the random module
    random.seed(seed)
    
    # Set the seed for numpy
    np.random.seed(seed)
    
    # Set the seed for PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Set the seed for PyTorch (GPU) if you are using CUDA
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    
    # Ensure deterministic behavior in PyTorch
    torch.backends.cudnn.deterministic = True  # This makes the computations deterministic
    torch.backends.cudnn.benchmark = False  # Disable auto-tuning for performance optimization
    
    # For reproducibility of other libraries like Python's `random`
    torch.random.manual_seed(seed)



In [4]:


class MLPBlock(nn.Module):
    def __init__(self, in_features=16, out_features=16, dropout_prob=0.0):
        super(MLPBlock, self).__init__()
        self.ff1 = nn.Linear(in_features, out_features)
        self.act = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = self.ff1(x)
        x = self.act(x)
        x = self.dropout(x)
        return x

class MLPModel(nn.Module):
    def __init__(self, input_size=16, hidden_size=16, num_blocks=3, dropout_prob=0.0):
        super(MLPModel, self).__init__()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.h = nn.ModuleList([MLPBlock(hidden_size, hidden_size, dropout_prob) for _ in range(num_blocks)])

    def forward(self, x):
        x = self.dropout(x)
        for layer in self.h:
            x = layer(x)
        return x

class MLPForClassification(nn.Module):
    def __init__(self, input_size=16, hidden_size=16, num_classes=2, num_blocks=3, dropout_prob=0.0):
        super(MLPForClassification, self).__init__()
        self.mlp = MLPModel(input_size, hidden_size, num_blocks, dropout_prob)
        self.score = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.mlp(x)
        x = self.score(x)
        return x



In [5]:
def randvec(n=50, lower=-0.5, upper=0.5):
    """
    Returns a random vector of length `n`. `w` is ignored.

    """
    return np.array([random.uniform(lower, upper) for i in range(n)])

def create_same_pair(embedding_size):
    vec=randvec(embedding_size)
    return vec,vec

def create_diff_pair(embedding_size):
    similar=True
    while similar:
        vec1=randvec(embedding_size)
        vec2=randvec(embedding_size)
        if not (vec1 == vec2).all():
            similar=False
    return vec1,vec2


def make_dataset_sample_model(embedding_size,variable1=False,variable2=False):
    First_pair_same=random.choice([True,False])
    if First_pair_same:
        First_pair=create_same_pair(embedding_size)
    else:
        First_pair=create_diff_pair(embedding_size)
        
    Second_pair_same=random.choice([True,False])
    if Second_pair_same:
        Second_pair=create_same_pair(embedding_size)
    else:
        Second_pair=create_diff_pair(embedding_size)
    
    modelinput=np.concatenate((First_pair, Second_pair), axis=None)
    if First_pair_same==Second_pair_same:
        label=1.0
    else:
        label=0.0
    return modelinput,label

def make_model_dataset(size,embedding_size):
    model_inputs=[]
    labels=[]
    for _ in tqdm(range(size)):
        model_input,label=make_dataset_sample_model(embedding_size)
        model_inputs.append(model_input)
        labels.append(label)
    return torch.tensor(model_inputs, dtype=torch.float32).to(DEVICE),torch.tensor(labels, dtype=torch.float32).to(DEVICE)



def make_dataset_sample_variable_intervention(embedding_size,variable1=False,variable2=False):
    First_pair_same=random.choice([True,False])
    if First_pair_same:
        First_pair=create_same_pair(embedding_size)
    else:
        First_pair=create_diff_pair(embedding_size)
    source_first_pair=[(np.zeros(embedding_size),np.zeros(embedding_size)),(np.zeros(embedding_size),np.zeros(embedding_size))]
    if variable1:
        First_pair_same=random.choice([True,False])
        if First_pair_same:
            source_first_pair=create_same_pair(embedding_size)
        else:
            source_first_pair=create_diff_pair(embedding_size)
        if random.choice([True,False]):
            source_first_pair=[source_first_pair,create_same_pair(embedding_size)]
        else:
            source_first_pair=[source_first_pair,create_diff_pair(embedding_size)]
        
    
    Second_pair_same=random.choice([True,False])
    if Second_pair_same:
        Second_pair=create_same_pair(embedding_size)
    else:
        Second_pair=create_diff_pair(embedding_size)
    source_second_pair=[(np.zeros(embedding_size),np.zeros(embedding_size)),(np.zeros(embedding_size),np.zeros(embedding_size))]
    if variable2:
        Second_pair_same=random.choice([True,False])
        if Second_pair_same:
            source_second_pair=create_same_pair(embedding_size)
        else:
            source_second_pair=create_diff_pair(embedding_size)
        if random.choice([True,False]):
            source_second_pair=[create_same_pair(embedding_size),source_second_pair]
        else:
            source_second_pair=[create_diff_pair(embedding_size),source_second_pair]
    
    
    modelinput=np.concatenate((First_pair, Second_pair), axis=None)
    source0=np.concatenate(source_first_pair, axis=None)
    source1=np.concatenate(source_second_pair, axis=None)
    if First_pair_same==Second_pair_same:
        label=1.0
    else:
        label=0.0
    return modelinput,label,source0,source1

def make_intervention_dataset_variable_intervention_all(size,embedding_size):
    intervention_data=[]
    for _ in tqdm(range(size)):
        variable1,variable2=random.choice([(True,False),(False,True),(True,True)])
        base,label,source0,source1=make_dataset_sample_variable_intervention(embedding_size,variable1=variable1,variable2=variable2)
        intervention_data.append({})
        intervention_data[-1]["base"]=torch.tensor(base, dtype=torch.float32)
        intervention_data[-1]["label"]=torch.tensor(label, dtype=torch.float32)
        intervention_data[-1]["sources"]=torch.tensor([source0,source1], dtype=torch.float32)
        intervention_data[-1]["intervention"]=[variable1,variable2]
        #print(intervention_data[-1])
    return intervention_data

def make_intervention_dataset_variable_intervention_first(size,embedding_size):
    intervention_data=[]
    for _ in tqdm(range(size)):
        base,label,source0,source1=make_dataset_sample_variable_intervention(embedding_size,variable1=True,variable2=False)
        intervention_data.append({})
        intervention_data[-1]["base"]=torch.tensor(base, dtype=torch.float32)
        intervention_data[-1]["label"]=torch.tensor(label, dtype=torch.float32)
        intervention_data[-1]["sources"]=torch.tensor([source0,source1], dtype=torch.float32)
        intervention_data[-1]["intervention"]=[True,False]
        #print(intervention_data[-1])
    return intervention_data




def make_dataset_sample_first_input_intervention(embedding_size):
    First_pair_same=random.choice([True,False])
    if First_pair_same:
        First_pair=create_same_pair(embedding_size)
    else:
        First_pair=create_diff_pair(embedding_size)
    
    Second_pair_same=random.choice([True,False])
    if Second_pair_same:
        Second_pair=create_same_pair(embedding_size)
    else:
        Second_pair=create_diff_pair(embedding_size)


    source=[]
    First_pair_same=random.choice([True,False])
    if First_pair_same: #equal or not in first variable
        if random.choice([True,False]): #Same source pair
            source.append((First_pair[1],First_pair[1]))
        else: #Different source pair
            new_vec=randvec(embedding_size)
            while (new_vec == First_pair[1]).all():
                new_vec=randvec(embedding_size)
            source.append((First_pair[1],new_vec))
            
    else:
        new_vec1=randvec(embedding_size)
        while (new_vec1 == First_pair[1]).all():
            new_vec1=randvec(embedding_size)
        if random.choice([True,False]): #Same source pair
            source.append((new_vec1,new_vec1))
        else: #Different source pair
            new_vec2=randvec(embedding_size)
            while (new_vec2 == new_vec1).all():
                new_vec2=randvec(embedding_size)
            source.append((new_vec1,new_vec2))

    if random.choice([True,False]):
        source.append(create_same_pair(embedding_size))
    else:
        source.append(create_diff_pair(embedding_size))
    
    
    modelinput=np.concatenate((First_pair, Second_pair), axis=None)
    source=np.concatenate(source, axis=None)
    if First_pair_same==Second_pair_same:
        label=1.0
    else:
        label=0.0
    return modelinput,label,source
    


def make_intervention_dataset_first_input_intervention(size,embedding_size):
    intervention_data=[]
    for _ in tqdm(range(size)):
        base,label,source=make_dataset_sample_first_input_intervention(embedding_size)
        intervention_data.append({})
        intervention_data[-1]["base"]=torch.tensor(base, dtype=torch.float32)
        intervention_data[-1]["label"]=torch.tensor(label, dtype=torch.float32)
        intervention_data[-1]["sources"]=torch.tensor([source], dtype=torch.float32)
        intervention_data[-1]["intervention"]=[True]
    return intervention_data




In [6]:
def train_model(model,batch_size = 1024,epochs=3):
    #Training
    X_train,y_train=make_model_dataset(1048576,4)
    # Create DataLoader
    
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Initialize model, loss function, and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch.squeeze().long())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
    return model

In [7]:
def test_model(model,batch_size = 1024):
    #Testing:
    X_test,y_test=make_model_dataset(10000,4)
    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in tqdm(test_loader):
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch.squeeze()).sum().item()
            total += y_batch.size(0)
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [8]:
def make_model(epochs=3):
    model = MLPForClassification()
    model.to(DEVICE)
    model=train_model(model,epochs=epochs)
    accuracy=test_model(model)
    return model,accuracy

In [9]:
class RotateLayer(torch.nn.Module):
    """A learnable linear transformation initialized as an orthogonal matrix."""

    def __init__(self, n, init_orth=True):
        """
        Args:
            n (int): Dimension of the square transformation matrix.
            init_orth (bool): If True, initializes the matrix with an orthogonal weight.
        """
        super().__init__()
        weight = torch.empty(n, n)  # Create an empty n x n matrix
        
        # We don't need initialization if loading from a pretrained checkpoint.
        # You can explore different initialization strategies if necessary, but this isn't our focus.
        if init_orth:
            torch.nn.init.orthogonal_(weight)
        
        self.weight = torch.nn.Parameter(weight, requires_grad=True)  # Learnable weight matrix

    def forward(self, x):
        """Applies the rotation matrix to the input tensor."""
        return torch.matmul(x.to(self.weight.dtype), self.weight)
        

class Transformation_Function(nn.Module):
    """Encapsulates the rotation transformation as a PyTorch module."""

    def __init__(self, embed_dim=16):
        """
        Args:
            embed_dim (int): The embedding dimension (size of the transformation matrix).
        """
        super(Transformation_Function, self).__init__()
        
        rotate_layer = RotateLayer(embed_dim)  # Initialize the rotation layer
        # Ensure the transformation remains an orthogonal matrix
        self.rotate_layer = torch.nn.utils.parametrizations.orthogonal(rotate_layer)

    def forward(self, x):
        """Applies the orthogonal transformation to the input tensor."""
        return self.rotate_layer(x)


class InverseTransformation_Function(nn.Module):
    """Computes the inverse of the given transformation function (phi)."""

    def __init__(self, transformation_function: Transformation_Function):
        """
        Args:
            transformation_function (Transformation_Function): The forward transformation function.
        """
        super(InverseTransformation_Function, self).__init__()
        self.transformation_function = transformation_function  # Store reference to the transformation

    def forward(self, x):
        """Applies the inverse transformation by transposing the orthogonal weight matrix."""
        weight_T = self.transformation_function.rotate_layer.weight.T  # Use matrix transpose as inverse
        return torch.matmul(x.to(weight_T.dtype), weight_T)

class phi_class:
    def __init__(self, phi,phi_inv,criterion,optimizer):
        self.phi=phi
        self.phi_inv=phi_inv
        self.criterion=criterion
        self.optimizer=optimizer



In [10]:
class Distributed_Alignment_Search:

    
    def __init__(self, 
                 Model,
                 Model_Layer,
                 Train_Data_Raw,
                 Test_Data_Raw,
                 Hidden_Layer_Size,
                 Variable_Dimensions,
                 Transformation_Class,
                 Device):
        self.Model=Model
        self.Model_Layer=Model_Layer
        self.Hidden_Layer_Size=Hidden_Layer_Size
        self.Variable_Dimensions=Variable_Dimensions
        self.Num_Varibales=len(self.Variable_Dimensions)
        self.Transformation_Class=Transformation_Class
        self.Device=Device

        self.mode_info=None
        self.base_activations=None
        self.source_activations=None

        
        self.Train_Dataset,self.Train_Sample_Number=self.Prepare_Dataset(Train_Data_Raw)
        self.Test_Dataset,self.Test_Samples_Number=self.Prepare_Dataset(Test_Data_Raw)

   
        self.Hook=self.register_intervention_hook(self.Model_Layer)
        

    def Prepare_Dataset(self,Raw_Dataset): 
        Clean_Dataset={}
        Clean_Dataset["base"]=[]
        Clean_Dataset["sources"]=[]
        for _ in range(self.Num_Varibales):
            Clean_Dataset["sources"].append([])
        Clean_Dataset["label"]=[]
        Clean_Dataset["intervention"]=[]

        #Actually adapted to the dataset I am using. Needs to be changed later:
        print("[SYST] Prepare Data")
        for ac_DP_pos in tqdm(range(len(Raw_Dataset))):
            #base input
            ac_DP=Raw_Dataset[ac_DP_pos]
            Clean_Dataset["base"].append(ac_DP["base"])

            #expected label after intervention
            Clean_Dataset["label"].append(ac_DP["label"])

            #source inputs
            for var_dim in range(ac_DP['sources'].shape[0]):
                Clean_Dataset["sources"][var_dim].append(ac_DP['sources'][var_dim])

            #intervention data
            Clean_Dataset["intervention"].append([])
            for Hidden_Dimension in range(self.Hidden_Layer_Size):
                dim_found=False
                for ac_dimensions_pos,ac_dimensions in enumerate(self.Variable_Dimensions):
                    if Hidden_Dimension in ac_dimensions:
                        dim_found=True
                        if ac_DP["intervention"][ac_dimensions_pos]:
                            Clean_Dataset["intervention"][-1].append(True)
                        else:
                            Clean_Dataset["intervention"][-1].append(False)
                        break
                if not dim_found:
                    Clean_Dataset["intervention"][-1].append(False)
            Raw_Dataset[ac_DP_pos]=None

        sample_number=len(Clean_Dataset["label"])
        Clean_Dataset["base"]=torch.stack(Clean_Dataset["base"]).to(self.Device)
        Clean_Dataset["label"]=torch.stack(Clean_Dataset["label"]).to(self.Device)
        for i in range(len(Clean_Dataset["sources"])):
            Clean_Dataset["sources"][i]=torch.stack(Clean_Dataset["sources"][i]).to(self.Device)
        Clean_Dataset["intervention"]=torch.tensor(Clean_Dataset["intervention"]).to(self.Device)
        return Clean_Dataset,sample_number
                        
        
    def register_intervention_hook(self, layer):
        def hook_fn(module, input, output):
            if self.mode_info[0] == "source":
                # Extract row indices (list)
                row_indices = self.mode_info[2]
                #print("SO",output)
                
                # Extract column indices (list)
                col_indices = self.Variable_Dimensions[self.mode_info[1]]
                #print("CI",col_indices)
                #print(self.source_activations.shape)
                #print("SB",self.source_activations)
                #print(output)
                
                # Get values from the transformation tensor
                transformed_values = self.Transformation_Class.phi(output.detach())[:, col_indices]
                #print("TV",transformed_values)
                
                # Assign these values to the corresponding positions in source_activations
                self.source_activations[torch.tensor(row_indices).unsqueeze(1), torch.tensor(col_indices).unsqueeze(0)] = transformed_values
                #print("SA",self.source_activations)
                #print("***")
            elif self.mode_info[0] == "intervene":
                result_tensor = self.Transformation_Class.phi(output.detach())
                #print("II",self.mode_info[1])
                #print("IBO",output)
                #print("IBT",result_tensor)
                #print("IBOR",self.Transformation_Class.phi_inv(result_tensor))
                result_tensor = torch.where(self.mode_info[1], self.source_activations, result_tensor)
                #print("IR",result_tensor)
                return self.Transformation_Class.phi_inv(result_tensor)
        
        return layer.register_forward_hook(hook_fn)

    
    def train_test(self,batch_size,epochs=1,train=True):

        self.Model.eval()
        for param in self.Model.parameters():
            param.requires_grad = False  # This freezes the weights

        if train:
            self.Transformation_Class.phi.train()
            for param in self.Transformation_Class.phi.parameters():
                param.requires_grad = True  # This unfreezes the weights
            data=self.Train_Dataset
            Sample_Indices=list(range(self.Train_Sample_Number))
        else:
            self.Transformation_Class.phi.eval()
            for param in self.Transformation_Class.phi.parameters():
                param.requires_grad = False  # This freezes the weights
            data=self.Test_Dataset
            Sample_Indices=list(range(self.Test_Samples_Number))
        
        for epoch in range(epochs):
            print("[SYST] Epoch",epoch)
            total_correct = 0
            total_samples = 0
            total_loss = 0
        
            #Make Batches
            random.shuffle(Sample_Indices)
            DAS_Train_Batches=self.chunk_list(Sample_Indices, batch_size)
            
            for ac_batch in tqdm(DAS_Train_Batches):

                if train: 
                    self.Transformation_Class.optimizer.zero_grad()
                
                #Prepare Source Activations
                self.source_activations = torch.zeros(len(ac_batch), self.Hidden_Layer_Size).to(self.Device)
                
                for ac_source_pos in range(len(data["sources"])):
                    #add info needed for optimization... no sense in running inputs who are not used:
                    used_source_indices=self.extract_sources_to_run(ac_source_pos,ac_batch,data)
                    if len(used_source_indices)>0:
                        #print("Hey")
                        self.mode_info=["source",ac_source_pos,used_source_indices]
                        ac_source=data["sources"][ac_source_pos][ac_batch][used_source_indices]
                        #print("SI",ac_source)
                        model(ac_source)
        
                #Intervention
                ac_base=data["base"][ac_batch]
                intervention_bools=data["intervention"][ac_batch]
                self.mode_info=["intervene",intervention_bools]
                #print("BI",ac_base)
                outputs=model(ac_base)
                labels=data["label"][ac_batch]
                #print("L",labels)
                if train:
                    loss = self.Transformation_Class.criterion(outputs, labels.squeeze().long())
                    loss.backward()
                    self.Transformation_Class.optimizer.step()
                    #print(self.Transformation_Class.phi.rotate_layer.weight)
                    total_loss += loss.item()

                else:
                    predictions = torch.argmax(outputs, dim=1) 
                    correct = (predictions.squeeze() == labels.squeeze()).sum().item()
                    total_correct += correct
                    total_samples += labels.size(0)
            
            if train:
                print(f"Epoch {epoch+1}, Loss: {total_loss / len(DAS_Train_Batches)}")
            else:     
                accuracy = total_correct / total_samples
                print(f"Test Accuracy: {accuracy * 100:.2f}%")
                return accuracy
                

    def extract_sources_to_run(self,which_variable,batch_indices,data):
        used_source_indices=[]
        for ip, i in enumerate(batch_indices):
            is_used=data["intervention"][i][self.Variable_Dimensions[which_variable][0]]
            if is_used:
                used_source_indices.append(ip)
        return used_source_indices

    
    def chunk_list(self, input_list, batch_size):
        return [input_list[i:i + batch_size] for i in range(0, len(input_list), batch_size)]

    
    def Cleanup(self):
        self.Hook.remove()

In [11]:
results=[]
for acseed in [4287, 3837, 9097, 2635, 5137, 6442, 5234, 4641, 8039, 2266]:
    results.append({})
    set_seed(acseed)
    model,accuracy=make_model(epochs=3)
    Layers=[]
    Layers.append(("Layer1",model.mlp.h[0]))
    Layers.append(("Layer2",model.mlp.h[1]))
    Layers.append(("Layer3",model.mlp.h[2]))
    inter_dims=[]
    
    """
    inter_dims.append([list(range(0,8)),list(range(8,16))])
    inter_dims.append([list(range(0,2)),list(range(2,4))])
    inter_dims.append([list(range(0,1)),list(range(1,2))])
    """
    inter_dims.append([list(range(0,8))])
    inter_dims.append([list(range(0,2))])
    inter_dims.append([list(range(0,1))])
    
    results[-1]["accuracy"]=accuracy
    for LayerName,Layer in Layers:
        results[-1][LayerName]={}
        for inter_dim in inter_dims:
            print(LayerName,":",inter_dim)
            #DAS_Train=make_intervention_dataset_variable_intervention_all(1280000,4)
            #DAS_Test=make_intervention_dataset_variable_intervention_all(10000,4)
            #DAS_Train=make_intervention_dataset_variable_intervention_first(1280000,4)
            #DAS_Test=make_intervention_dataset_variable_intervention_first(10000,4)
            DAS_Train=make_intervention_dataset_first_input_intervention(1280000,4)
            DAS_Test=make_intervention_dataset_first_input_intervention(10000,4)
    
            #Initialize transformation function
            p=Transformation_Function(16)
            p.to(DEVICE)
            p_inverse=InverseTransformation_Function(p)
            
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(p.parameters(), lr=0.001)
            
            phi=phi_class(p,p_inverse,criterion,optimizer)
    
            
    
            DAS_Experiment=Distributed_Alignment_Search(Model=model,
                                                        Model_Layer=Layer,
                                                        Train_Data_Raw=DAS_Train,
                                                        Test_Data_Raw=DAS_Test,
                                                        Hidden_Layer_Size=16,
                                                        Variable_Dimensions=inter_dim,
                                                        Transformation_Class=phi,
                                                        Device=DEVICE)
    
            DAS_Experiment.train_test(batch_size=6400,
                                      epochs=10,
                                      train=True)
    
            accuracy=DAS_Experiment.train_test(batch_size=6400,
                                               train=False)
            
            results[-1][LayerName][str(inter_dim)]=accuracy
            DAS_Experiment.Cleanup()
            DAS_Experiment=None
            print(results)

100%|█████████████████████████████████████████████| 1048576/1048576 [00:10<00:00, 97764.23it/s]
/tmp/ipykernel_33726/3160764943.py:49: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  return torch.tensor(model_inputs, dtype=torch.float32).to(DEVICE),torch.tensor(labels, dtype=torch.float32).to(DEVICE)
100%|█████████████████████████████████████████████████████| 1024/1024 [00:09<00:00, 110.35it/s]


Epoch 1, Loss: 0.3913447721170087


100%|█████████████████████████████████████████████████████| 1024/1024 [00:09<00:00, 113.47it/s]


Epoch 2, Loss: 0.02623706526355818


100%|█████████████████████████████████████████████████████| 1024/1024 [00:08<00:00, 113.83it/s]


...

Epoch 9, Loss: 5.0634840083122254
[SYST] Epoch 9


100%|████████████████████████████████████████████████████████| 200/200 [00:20<00:00,  9.99it/s]


Epoch 10, Loss: 5.063301341533661
[SYST] Epoch 0


100%|████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.23it/s]

Test Accuracy: 49.53%
[{'accuracy': 0.9983, 'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5013, '[[0, 1]]': 0.5039, '[[0]]': 0.5082}, 'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5037, '[[0, 1]]': 0.5032, '[[0]]': 0.5075}, 'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4925, '[[0, 1]]': 0.5037, '[[0]]': 0.5081}}, {'accuracy': 0.9983, 'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5012, '[[0, 1]]': 0.5059, '[[0]]': 0.4956}, 'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5027, '[[0, 1]]': 0.4957, '[[0]]': 0.493}, 'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4966, '[[0, 1]]': 0.4996, '[[0]]': 0.5071}}, {'accuracy': 0.9906, 'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4964, '[[0, 1]]': 0.5118, '[[0]]': 0.5024}, 'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4993, '[[0, 1]]': 0.4958, '[[0]]': 0.5061}, 'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4949, '[[0, 1]]': 0.5015, '[[0]]': 0.4977}}, {'accuracy': 0.9955, 'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5024, '[[0, 1]]': 0.5025, '[[0]]': 0.4974}, 'Layer2': {'[[0, 1, 2

In [12]:
results

[{'accuracy': 0.9983,
  'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5013,
   '[[0, 1]]': 0.5039,
   '[[0]]': 0.5082},
  'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5037,
   '[[0, 1]]': 0.5032,
   '[[0]]': 0.5075},
  'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4925,
   '[[0, 1]]': 0.5037,
   '[[0]]': 0.5081}},
 {'accuracy': 0.9983,
  'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5012,
   '[[0, 1]]': 0.5059,
   '[[0]]': 0.4956},
  'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5027,
   '[[0, 1]]': 0.4957,
   '[[0]]': 0.493},
  'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4966,
   '[[0, 1]]': 0.4996,
   '[[0]]': 0.5071}},
 {'accuracy': 0.9906,
  'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4964,
   '[[0, 1]]': 0.5118,
   '[[0]]': 0.5024},
  'Layer2': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4993,
   '[[0, 1]]': 0.4958,
   '[[0]]': 0.5061},
  'Layer3': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.4949,
   '[[0, 1]]': 0.5015,
   '[[0]]': 0.4977}},
 {'accuracy': 0.9955,
  'Layer1': {'[[0, 1, 2, 3, 4, 5, 6, 7]]': 0.5024,
   '

In [13]:
import json
with open('results.json', 'w') as f:
    json.dump(results, f)
